In [1]:
import numpy as np
import pingouin as pg
import pandas as pd
import rasterio


In [ ]:
tif1_path = r"D:\paper01\VPD&SM(M03)\GIMMS_NIRV\partial_cor\VPD\impact_factor\vpd_loss.tif"
tif2_path = r"D:\paper01\VPD&SM(M03)\GIMMS_NIRV\partial_cor\VPD\impact_factor\dem.tif"
tif3_path = r"D:\paper01\VPD&SM(M03)\GIMMS_NIRV\partial_cor\VPD\impact_factor\t2m_Clip.tif"
tif4_path = r"D:\paper01\VPD&SM(M03)\GIMMS_NIRV\partial_cor\VPD\impact_factor\Intensity.tif"

# 数据读取和预处理
def read_tif(path):
    with rasterio.open(path) as tif:
        data = tif.read(1).astype(float)
        data[data < -9999] = np.nan  # 或根据实际 nodata 处理
        return data

data1 = read_tif(tif1_path)
data2 = read_tif(tif2_path)
data3 = read_tif(tif3_path)
data4 = read_tif(tif4_path)
# data5 = read_tif(tif5_path)

window_size = 7
stride = 1
result_rows = data1.shape[0] - window_size + 1
result_cols = data1.shape[1] - window_size + 1

result = np.full((result_rows, result_cols), np.nan)
p_values = np.full((result_rows, result_cols), np.nan)

for i in range(result_rows):
    for j in range(result_cols):
        w1 = data1[i:i+window_size, j:j+window_size].flatten()
        w2 = data2[i:i+window_size, j:j+window_size].flatten()
        w3 = data3[i:i+window_size, j:j+window_size].flatten()
        w4 = data4[i:i+window_size, j:j+window_size].flatten()

        # 判断有效值数量
        if any(np.count_nonzero(~np.isnan(w)) < 3 for w in [w1, w2, w3, w4]):
            continue

        df = pd.DataFrame({'data1': w1, 'data2': w2, 'data3': w3, 'data4': w4}).dropna()

        if len(df) < 3:
            continue

        # 计算偏相关：data1 ~ data3, 控制其他变量
        res = pg.partial_corr(data=df, x='data2', y='data1', covar=['data4', 'data3'])

        r = res['r'].values[0]
        p = res['p-val'].values[0]

        # 去除偏相关系数异常值
        if abs(r) > 0.8:
            continue

        result[i, j] = r
        p_values[i, j] = p

# 掩膜显著性水平
filtered_result = np.copy(result)
filtered_result[p_values > 0.05] = np.nan



In [ ]:
filtered_result = -filtered_result
result = -result

In [ ]:
import rasterio
from rasterio.transform import Affine
from rasterio.transform import from_origin
# 保存结果到 GeoTIFF 文件
output_path_filtered = r"D:\paper01\TEST\VPD&dem_unfilter.tif"

# 假设你有一个 GeoTIFF 文件作为源文件
source_tif_path = r"D:\paper01\VPD&SM(M03)\GIMMS_NIRV\partial_cor\VPD\impact_factor\VPD&dem_filter.tif"

# 读取源文件的地理信息
with rasterio.open(source_tif_path) as src_tif:
    profile = src_tif.profile


# 保存结果
with rasterio.open(
    output_path_filtered,
    'w',
    **profile  # 使用源文件的地理信息
) as dst:
    dst.write(filtered_result, 1)